In [1]:
##Load required libraries
library(randomForest)
library(dplyr)
library(ggplot2)
library(reshape2)
library(pROC)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'dplyr'


The following object is masked from 'package:randomForest':

    combine


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Warning message:
"package 'ggplot2' was built under R version 4.4.3"

Attaching package: 'ggplot2'


The following object is masked from 'package:randomForest':

    margin


Warning message:
"package 'pROC' was built under R version 4.4.3"
Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




In [2]:
##Read CSV file
whole_data <- read.csv("C8_Heart_featuremeta.csv")

#Subset DataFrame to include only specific cohort, if needed. Remove this line and adjust code accordingly.
subset_data <- whole_data[whole_data$`ATTRIBUTE_Cohort` == 1, ]
  

##Prints subsetted df.
dim(subset_data)

#G1-G3 Random Forest. 2000 trees.

subset_data_G1_G3 <- subset_data %>%
  filter(ATTRIBUTE_Group_number %in% c('Group1', 'Group3'))

subset_data_G1_G3

X_G1_G3 <- subset_data_G1_G3[, 14:3414]
y_G1_G3 <- as.factor(subset(subset_data, ATTRIBUTE_Group_number %in% c("Group1", "Group3"))$ATTRIBUTE_Group_number)
X_G3_G1 <- subset_data_G1_G3[, 14:3414]
y_G3_G1 <- as.factor(subset(subset_data, ATTRIBUTE_Group_number %in% c("Group3", "Group1"))$ATTRIBUTE_Group_number)

[1]  164 3414

sampleid,ATTRIBUTE_TYPE_tissue,ATTRIBUTE_Type,Tube_Number,ATTRIBUTE_Cohort,Mouse.ID,ATTRIBUTE_Group_number,ATTRIBUTE_Euthanasia_date,ATTRIBUTE_DPI,ATTRIBUTE_organ,⋯,X749.5031_3.04_5615,X611.053_2.47_5616,X502.8139_2.39_5617,X550.8507_2.49_5618,X539.8441_2.48_5619,X581.7023_2.46_5620,X232.1907_2.41_5621,X724.4499_3.17_5622,X204.0863_6.12_5623,X760.5834_2.86_5624
<chr>,<chr>,<chr>,<int>,<int>,<int>,<chr>,<int>,<int>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Upper_right_1_434_905_1_44943_210_heart_P268_Y_A1,Sample_heart,Sample,434,1,905,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_435_906_1_44943_210_heart_P268_Y_A2,Sample_heart,Sample,435,1,906,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_436_907_1_44943_210_heart_P268_Y_A3,Sample_heart,Sample,436,1,907,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_437_908_1_44943_210_heart_P268_Y_A4,Sample_heart,Sample,437,1,908,Group1,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_441_793_3_44943_210_heart_P268_Y_A8,Sample_heart,Sample,441,1,793,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_442_794_3_44943_210_heart_P268_Y_A9,Sample_heart,Sample,442,1,794,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_443_795_3_44943_210_heart_P268_Y_A10,Sample_heart,Sample,443,1,795,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
Upper_right_1_444_796_3_44943_210_heart_P268_Y_A11,Sample_heart,Sample,444,1,796,Group3,44943,210,heart,⋯,0,0,0,0,0,0,0,0,0,0
middle_top_1_380_897_1_44936_203_heart_P266_B_A1,Sample_heart,Sample,380,1,897,Group1,44936,203,heart,⋯,0,0,0,0,0,0,0,0,0,0


In [3]:
n_runs <- 2           #Number of times to run RF
top_n <- 100            #Top N features to keep from each run
set.seed(123)          #Seed set for reproducibility
feature_counts <- list()
importance_list <- list()

In [4]:
#G1 vs G3, ntrees=2000, trained 100x, frequency >70%, imp cutoff >=1.5

#Store top features and importance values from each run
for (i in 1:n_runs) {
  rf_model <- randomForest(X_G1_G3, y_G1_G3, ntree = 1000, importance = TRUE)
  importance_vals <- importance(rf_model, type = 1)[,1]  # MDA
  importance_list[[i]] <- importance_vals
  
  top_features <- names(sort(importance_vals, decreasing = TRUE))[1:top_n]
  feature_counts[[i]] <- top_features
}

#Count frequency of top features
all_top_features <- unlist(feature_counts)
feature_freq <- table(all_top_features)
feature_freq <- sort(feature_freq, decreasing = TRUE)

#Convert frequency table to df
consensus_df <- data.frame(
  feature = names(feature_freq),
  count = as.integer(feature_freq),
  frequency = as.integer(feature_freq) / n_runs
)

#Combine importance values across runs
all_features <- unique(unlist(lapply(importance_list, names)))
importance_matrix <- sapply(importance_list, function(x) {
  x[all_features]  #get importance values for all features, NA if missing
})
rownames(importance_matrix) <- all_features

#Compute mean importance per feature
mean_importance <- rowMeans(importance_matrix, na.rm = TRUE)
importance_df <- data.frame(feature = names(mean_importance), mean_importance = mean_importance)

#Merge with consensus_df
merged_df <- merge(consensus_df, importance_df, by = "feature")

#Filter: at least 70% frequency and at least the designated importance
filtered <- subset(merged_df, frequency > 0.7 & mean_importance >= 1.5)

print(filtered)

write.csv(filtered, "1.5_cutoff_70freq_G1_G3.csv", row.names = FALSE)

                feature count frequency mean_importance
2    X116.0707_0.27_554     2         1        1.872817
3    X119.0853_2.86_358     2         1        2.572168
8    X153.0405_0.36_315     2         1        4.272855
11   X161.0919_0.32_127     2         1        3.222473
14   X166.0861_0.31_265     2         1        1.572096
15   X179.0622_2.25_378     2         1        2.085339
17   X199.0478_0.3_1479     2         1        1.832534
18   X211.0785_2.88_399     2         1        2.519020
23  X227.1098_2.61_1216     2         1        2.313688
24     X229.1545_0.3_73     2         1        1.916310
26   X251.0363_0.26_348     2         1        1.843640
27   X255.6452_3.23_247     2         1        1.616609
28   X256.1902_2.85_154     2         1        2.908542
32    X268.1036_0.34_74     2         1        3.551052
37    X285.0206_2.41_49     2         1        2.699962
43   X307.1721_2.83_268     2         1        3.102040
49   X325.2191_3.04_186     2         1        3

In [5]:
#G3 vs G1, ntrees=2000, trained 100x, frequency >70%, imp cutoff >=1.5

#Store top features and importance values from each run
for (i in 1:n_runs) {
  rf_model <- randomForest(X_G1_G3, y_G1_G3, ntree = 2000, importance = TRUE)
  importance_vals <- importance(rf_model, type = 1)[,1]  # MDA
  importance_list[[i]] <- importance_vals
  
  top_features <- names(sort(importance_vals, decreasing = TRUE))[1:top_n]
  feature_counts[[i]] <- top_features
}

#Count frequency of top features
all_top_features <- unlist(feature_counts)
feature_freq <- table(all_top_features)
feature_freq <- sort(feature_freq, decreasing = TRUE)

#Convert frequency table to df
consensus_df <- data.frame(
  feature = names(feature_freq),
  count = as.integer(feature_freq),
  frequency = as.integer(feature_freq) / n_runs
)

#Combine importance values across runs
all_features <- unique(unlist(lapply(importance_list, names)))
importance_matrix <- sapply(importance_list, function(x) {
  x[all_features]  #get importance values for all features, NA if missing
})
rownames(importance_matrix) <- all_features

#Compute mean importance per feature
mean_importance <- rowMeans(importance_matrix, na.rm = TRUE)
importance_df <- data.frame(feature = names(mean_importance), mean_importance = mean_importance)

#Merge with consensus_df
merged_df <- merge(consensus_df, importance_df, by = "feature")

#Filter: at least 70% frequency and at least the designated importance
filtered <- subset(merged_df, frequency > 0.7 & mean_importance >= 1.5)

print(filtered)

write.csv(filtered, "1.5_cutoff_70freq_G3_G1.csv", row.names = FALSE)

                feature count frequency mean_importance
2    X116.0707_0.27_554     2         1        2.249308
4    X119.0853_2.86_358     2         1        3.435755
8      X132.0767_0.27_3     2         1        1.878011
11   X153.0405_0.36_315     2         1        5.097330
12   X153.0545_2.34_605     2         1        1.647484
15   X161.0919_0.32_127     2         1        3.934631
16   X166.0861_0.31_265     2         1        2.607876
18   X179.0622_2.25_378     2         1        3.460637
21   X199.0478_0.3_1479     2         1        2.497626
23  X213.2323_2.69_5486     2         1        1.894263
27   X220.9677_0.24_366     2         1        2.689300
28   X226.0805_2.25_558     2         1        2.561287
29  X227.1098_2.61_1216     2         1        2.862367
33   X251.0363_0.26_348     2         1        1.828723
34   X255.6452_3.23_247     2         1        2.019209
35   X256.1902_2.85_154     2         1        3.227590
38    X268.1036_0.34_74     2         1        3